In [27]:
# import the required libraries

import os
import re
import csv
import pandas as pd
import numpy as np


In [28]:
# Set variables for the project (i.e. the input location of the file to be processed and the output location) )

folderlink = '..//data//'
input_folder = 'input//'
input_file = os.path.join(folderlink+input_folder, 'boekzaallijst_27072023.csv')
folder_output = 'output//'
output_csv = folderlink+folder_output+'clerus_boekzaal.csv'
input_file_parent = folderlink+folder_output+"parent_data.csv"
input_file_minister = folderlink+folder_output+"minister_info.csv"

# Panda settings for showing data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [29]:
years_to_integers = {'Jaar (Begin) Rol': pd.Int64Dtype(),'Jaar Beroepen': pd.Int64Dtype(), }
boekzaallijst = pd.read_csv(input_file, sep=';', dtype=years_to_integers, encoding='utf-8')

In [30]:
boekzaallijst.head(20)

,Nr_BZ,Achternaam_BZ,Tussenvoegsel_BZ,Voornaam_BZ,Voorletter_BZ,Variate_Achternaam_BZ,Variate_Voornaam_BZ,Variatie_Voorletter_BZ,Family name,Toevoeging,Classis,Rol,Rol plaats,Jaar (Begin) Rol,Periode (eind) Rol,Jaar 2 en verder rol,Periode 2 (en verder) rol,Jaar (eind) rol,Periode (eind) rol,Jaar Beroepen,Periode (Beroepen),Rol.1,Bestemming,Toevoeging plaats bestemming,Aantekening bij bestemming,Overleden,Jaar overlijden,Extra functie,Locatie xtra functie,Jaar begin xtra functie,Periode xtra functie,Unnamed: 31,Periode (einde),Aantekening xtra functie
0,1,Aaken,van,NaN,J.,NaN,NaN,J.H.,NaN,NaN,13,Proponent,NaN,1815,AB,NaN,NaN,NaN,NaN,1816,A,Predikant,St. Michiels Gestel en Gemunde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Aalburg,van,Joannes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,1743,B,Predikant,Oudkarspel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Aardenburg,van,Buwama,C.,NaN,NaN,NaN,NaN,NaN,52,Proponent,NaN,1716,A(B),NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Aardse,NaN,Albertus,NaN,NaN,NaN,NaN,NaN,NaN,44,Proponent,NaN,1757,B,1758,AB,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,Aartsen,NaN,Albertus,NaN,NaN,NaN,NaN,NaN,NaN,40,Proponent,NaN,1766,AB,1767,AB,NaN,NaN,1776,B,Predikant,Randwijk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,Aartsen,NaN,Antonius,NaN,Aartzen,NaN,NaN,NaN,NaN,41,Proponent,NaN,1742,A,NaN,NaN,NaN,NaN,1742,B,Predikant,Zunderdorp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,Aartze,NaN,Lambertus,NaN,NaN,NaN,NaN,Ant. Fil.,NaN,26,Proponent,NaN,1773,B,1774,A,NaN,NaN,1774,B,Predikant,Voorhout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,Abbema,NaN,Andries,NaN,NaN,NaN,NaN,NaN,NaN,36,Proponent,NaN,1793,AB,1794,AB,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,Abbenbroek,NaN,Wilhelmus,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,1743,A,Predikant,Oost-Indie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,Abbering,NaN,Hillebrandus,NaN,NaN,NaN,NaN,NaN,NaN,45,Proponent,NaN,1783,A,NaN,NaN,NaN,NaN,1783,B,Predikant,Oterdum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Define a function to get the first letter from 'Name' or 'Initial'
def get_first_letter(row):
    name_letter = row['Voornaam_BZ'][0] if pd.notnull(row['Voornaam_BZ']) else None
    initial_letter = row['Voorletter_BZ'][0] if pd.notnull(row['Voorletter_BZ']) else None
    return name_letter or initial_letter


In [32]:
# Create the new field 'First_Letter'
boekzaallijst['first_letter'] = boekzaallijst.apply(get_first_letter, axis=1)

In [33]:
fil_boekzaallijst = boekzaallijst.dropna(subset=['Jaar Beroepen'])

In [34]:
fil_boekzaallijst.head()

,Nr_BZ,Achternaam_BZ,Tussenvoegsel_BZ,Voornaam_BZ,Voorletter_BZ,Variate_Achternaam_BZ,Variate_Voornaam_BZ,Variatie_Voorletter_BZ,Family name,Toevoeging,Classis,Rol,Rol plaats,Jaar (Begin) Rol,Periode (eind) Rol,Jaar 2 en verder rol,Periode 2 (en verder) rol,Jaar (eind) rol,Periode (eind) rol,Jaar Beroepen,Periode (Beroepen),Rol.1,Bestemming,Toevoeging plaats bestemming,Aantekening bij bestemming,Overleden,Jaar overlijden,Extra functie,Locatie xtra functie,Jaar begin xtra functie,Periode xtra functie,Unnamed: 31,Periode (einde),Aantekening xtra functie,first_letter
0,1,Aaken,van,NaN,J.,NaN,NaN,J.H.,NaN,NaN,13,Proponent,NaN,1815,AB,NaN,NaN,NaN,NaN,1816,A,Predikant,St. Michiels Gestel en Gemunde,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J
1,2,Aalburg,van,Joannes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,1743,B,Predikant,Oudkarspel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,J
4,5,Aartsen,NaN,Albertus,NaN,NaN,NaN,NaN,NaN,NaN,40,Proponent,NaN,1766,AB,1767,AB,NaN,NaN,1776,B,Predikant,Randwijk,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
5,6,Aartsen,NaN,Antonius,NaN,Aartzen,NaN,NaN,NaN,NaN,41,Proponent,NaN,1742,A,NaN,NaN,NaN,NaN,1742,B,Predikant,Zunderdorp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
6,7,Aartze,NaN,Lambertus,NaN,NaN,NaN,NaN,Ant. Fil.,NaN,26,Proponent,NaN,1773,B,1774,A,NaN,NaN,1774,B,Predikant,Voorhout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L


In [35]:
fil_boekzaallijst['boekzaallink'] = fil_boekzaallijst['first_letter'].astype(str) + '_' + fil_boekzaallijst['Achternaam_BZ'].astype(str) + '_' + fil_boekzaallijst['Jaar Beroepen'].astype(str).str.replace(' ', '')

C:\Windows\Temp\ipykernel_4272\3277643486.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_boekzaallijst['boekzaallink'] = fil_boekzaallijst['first_letter'].astype(str) + '_' + fil_boekzaallijst['Achternaam_BZ'].astype(str) + '_' + fil_boekzaallijst['Jaar Beroepen'].astype(str).str.replace(' ', '')


In [36]:
years_to_integers = {'minister_year': pd.Int64Dtype() }
minister = pd.read_csv(input_file_minister, sep=';', dtype=years_to_integers, encoding='utf-8')

In [37]:
minister.head()

,clerus_id,first_letter,minister,join_name,original_input,minister_year,accu_year_minster
0,1,J,Oudkarspel,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",1743,NaN
1,4,W,Aardenburg,Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",1695,NaN
2,2,C,Kalslagen ber,Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",1717,NaN
3,3,G,Vuren en Dalem,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1704,NaN
4,3,G,Sommelsdijk,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1706,NaN


In [38]:
minister['join_name_1'] = minister['join_name'].str.split().str[0]
minister['drc_link'] = minister['first_letter'].astype(str) + '_' +minister['join_name_1'].astype(str) + '_' + minister['minister_year'].astype(str).str.replace(' ', '')

In [39]:
minister.head()

,clerus_id,first_letter,minister,join_name,original_input,minister_year,accu_year_minster,join_name_1,drc_link
0,1,J,Oudkarspel,Aalburg Johannes van,"Aalburg, van Johannes Geb. Zierikzee ca. 1717 pred. Oudkarspel 30 juni 1743, overl. 14 maart 1777.<1>",1743,NaN,Aalburg,J_Aalburg_1743
1,4,W,Aardenburg,Aalst Wilhelmus,"Aalst Wilhelmus Gedoopt Biggekerke 5 jan. 1664 pred. Aardenburg 22 mei 1695, overl. 19 dec. 1700.<4>",1695,NaN,Aalst,W_Aalst_1695
2,2,C,Kalslagen ber,Aalst Cornelius van,"Aalst, van Cornelius Geb. Castricum ca. 1686 ambassadepred. in Parijs maart tot dec. 1715 pred. Kalslagen ber. 21 febr. 1717, emer. 1751 overl. Amsterdam 27 aug. 1756.<2>",1717,NaN,Aalst,C_Aalst_1717
3,3,G,Vuren en Dalem,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1704,NaN,Aalst,G_Aalst_1704
4,3,G,Sommelsdijk,Aalst Gerardus van,"Aalst, van Gerardus Geb. xxx sept. 1678 pred. Vuren en Dalem 10 aug. 1704, Sommelsdijk 13 juni 1706, WestZaandam 4 aug. 1715, emer. 1755 overl. 29 juni 1759.<3>",1706,NaN,Aalst,G_Aalst_1706


In [40]:
boekzaal_minister_inner = pd.merge(fil_boekzaallijst, minister, left_on='boekzaallink', right_on='drc_link', how='inner')
boekzaal_minister_inner.to_csv(folderlink+folder_output+'boekzaal_minister_year_inner.csv', sep=';', encoding='utf-8', index=False)

In [44]:
boekzaal_minister_left = pd.merge(fil_boekzaallijst, minister, left_on='boekzaallink', right_on='drc_link', how='left')
boekzaal_minister_left.to_csv(folderlink+folder_output+'boekzaal_minister_year_left.csv', sep=';', encoding='utf-8', index=False)